In [1]:
import os
print(os.getcwd())
if os.getcwd().endswith("mri_data"):
    os.chdir("..")
    os.chdir("..")

from src.modelling import DeepModel_Trainer
from src.augmentation import Transformer,None_Transform
from src.data_modules import DataModule,ImagesDataset
from torch import nn
import torch
from torchvision import transforms,models
import nibabel as nib
import numpy as np

/mnt/batch/tasks/shared/LS_root/mounts/clusters/ksajazwi-cpu/code/Users/pro5d-classification-prolactinoma/modelling/mri_data


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/cuda/__init__.py:740: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [30]:
%env "WANDB_NOTEBOOK_NAME" "ccv1_efficient_net"
%env WANDB_SILENT=True

base_transformer = Transformer(
    None_Transform(), "select_roi"
)
# %%
def base_mlp():
    model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(
        10000, 1
    )) 
    return model

env: "WANDB_NOTEBOOK_NAME"="ccv1_efficient_net"
env: WANDB_SILENT=True


In [36]:
first = np.array([1,0,1,0])
second = np.array([1,0,0,1])

In [18]:
check_data_modul = DataModule(base_transformer)
check_data_modul.prepare_data(0)

In [32]:
data = check_data_modul.val.data

mri = check_data_modul.val.load_mri(data.iloc[10]["MRI_Case_ID"])
visualize_slices([mri])

interactive(children=(IntSlider(value=0, description='mri', max=0), IntSlider(value=7, description='slice_numb…

7700506

In [ ]:
efficientnet = DeepModel_Trainer(DataModule(base_transformer), base_mlp)
efficientnet.train_model("Baseline MLP", "base_mlp", num_epochs=6, test_model=False,batchsize_train_data=128,use_mri_images=True,use_tabular_data=False)